In [1]:
import torch
import numpy as np
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import librosa
import librosa.display
import IPython.display

In [2]:
MFCC_X = 20
MFCC_Y = 44

In [3]:
def _path(loc, directory, filename, redundant, format_):
  delimiter = '\\'
  path = (loc + delimiter + directory + delimiter + filename + redundant + format_)
  return _path

def _normalize(mfcc):
  return (mfcc - np.min(mfcc)) / (np.max(mfcc) - np.min(mfcc))

def _fit_size(arr, size):
  zeros = np.zeros((len(arr), size - len(arr[0])))
  arr = np.append(arr, zeros, axis = 1)
  arr.astype('float32')
  return arr

def _mfcc(path):
  y, sr = librosa.load(path)
  mfcc = librosa.feature.mfcc(y = y, sr = sr)
  mfcc = _fit_size(mfcc, MFCC_Y)
  return _normalize(mfcc)

In [4]:
class CustomDataset(Dataset):
  def __init__(self, train = True):
    label_list = [0, 1, 2, 3]
    self.label_word = ['yes', 'no', 'on', 'off']
    self.train = train
    num_data = 100
    num_train = 90
    num_test = 10

    if self.train == True:
      self.train_data = []
      self.train_label = []

      print("\n\n==== Train Data : ")
      for item in label_list:
        print("===", end = "")
        for i in range(1, num_train + 1):
          path = _path(loc = '.\Data\Audio', directory = self.label_word[item], filename = self.label_word[item], redundant = '(' + str(i) + ')', format_ = '.wav')
          mfcc = _mfcc(path)
          self.train_label = np.array(self.train_label)
          self.train_data = np.concatenate(self.train_data)
          self.train_data = self.train_data.reshape(num_train * 4, MFCC_X, MFCC_Y) #3600개에 (20, 44) mfcc

    else:
      self.test_data = []
      self.test_label = []

      print("\n\n=== Test Data: ")
      for item in label_list:
        print("===", end = "")
        for i in range(num_train + 1, num_data + 1):
          path = _path(loc = '.\Data\Audio', directory = self.label_word[item], filename = self.label_word[item], redundant = '(' + str(i) + ')', format_ = '.wav')
          mfcc = mfcc(path)
          self.test_data.append(mfcc)
          self.test_label.append(item)

      self.test_label = np.array(self.test_label)
      self.test_data = np.concatenate(self.test_data)
      self.train_data = self.train_data.reshape(num_train * 4, MFCC_X, MFCC_Y) #3600개 (20, 44) mfcc

In [5]:
train_dataset = CustomDataset(train = True)
test_dataset = CustomDataset(train = False)

train_loader = DataLoader(train_dataset, batch_size=10, shuffle = True, num_workers = 0, drop_last = True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle = False, num_workers = 0, drop_last = True)



==== Train Data : 
===

TypeError: Invalid file: <function _path at 0x000001EB4B7C89D0>